The ReAct (Reasoning + Acting) Agent is a framework that combines reasoning and acting in AI agents. It enables the agent to think step by step, retrieve information, and interact with tools to accomplish tasks.

In [76]:
import os
from dotenv import load_dotenv
load_dotenv()


GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
GROQ_API_KEY=os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT=os.getenv("LANGCHAIN_PROJECT")

GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
GROQ_API_KEY=os.getenv("GROQ_API_KEY")
LANGSMITH_API_KEY=os.getenv("LANGSMITH_API_KEY")
LANGSMITH_PROJECT=os.getenv("LANGSMITH_PROJECT")
os.environ["LANGSMITH_TRACING"]="true"
os.environ["LANGSMITH_ENDPOINT"]="https://api.smith.langchain.com"


In [8]:
from langchain_groq import ChatGroq
llm=ChatGroq(model_name="llama-3.1-8b-instant")
llm.invoke("hi").content

# from langchain_google_genai import ChatGoogleGenerativeAI
# llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro")
# llm.invoke("hi").content

'How can I assist you today?'

In [9]:
message=[{"role":"system","content":"you are a helpful assistant"},
         {"role":"user","content":"hi how are you?"}]

In [10]:
result = llm.invoke(message)
print(result.content)

I'm doing well, thank you for asking. I'm a helpful assistant, and I'm here to assist you with any questions or tasks you may have. How can I help you today?


In [11]:
class Chatbot:
    def __init__(self,system=""):
        self.system=system
        self.message=[]
        if self.system:
            self.message.append({"role":"system","content":system})
    def __call__(self,message):
        self.message.append({"role":"user","content":message})
        result=self.execute()
        self.message.append({"role":"assistant","content":result})
        return result
        
    def execute(self):
        llm=ChatGroq(model_name="llama-3.1-8b-instant")
        # llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro")
        result = llm.invoke(self.message)
        return result.content
        

In [12]:
bot=Chatbot(system="you are helpful assistant")

In [13]:
bot("hi how are you?")

"I'm doing well, thank you for asking. I'm a helpful assistant here to provide information, answer questions, and assist with tasks to the best of my abilities. How can I help you today?"

In [14]:
bot.message

[{'role': 'system', 'content': 'you are helpful assistant'},
 {'role': 'user', 'content': 'hi how are you?'},
 {'role': 'assistant',
  'content': "I'm doing well, thank you for asking. I'm a helpful assistant here to provide information, answer questions, and assist with tasks to the best of my abilities. How can I help you today?"}]

In [15]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop your output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.


Your available actions are:
calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point
syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

simon_blog_search:
e.g. simon_blog_search: Python
Search Simon's blog for that term

Example session:
Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:
Observation: France is a country. The capital is Paris.

You then output:
Answer: The capital of France is Paris

Please Note: if you get basic conversation questions like "hi","hello","how are you?",\n
you have to answer "hi","hello","i am good".
""".strip()

In [16]:
prompt2 = """You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop, you output an Answer related to fitness, health, or exercise.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 1800 * 0.2
Runs a calculation (e.g., calorie intake, BMI, workout progress) and returns the result.

fitness_wiki:
e.g. fitness_wiki: Benefits of Cardio
Returns a summary of fitness-related information from a fitness knowledge base (Wikipedia or similar).

exercise_plan:
e.g. exercise_plan: Beginner full-body workout
Returns a recommended workout plan based on the type of exercise or user level.

nutrition_info:
e.g. nutrition_info: Protein sources for vegetarians
Looks up nutritional information or suggestions for diets.

Example session:
Question: What are the benefits of cardio exercises?
Thought: I should look up information about the benefits of cardio exercises.
Action: fitness_wiki: Benefits of Cardio
PAUSE

You will be called again with this:

Observation: Cardio exercises improve cardiovascular health, increase lung capacity, and aid in weight loss.

You then output:

Answer: Cardio exercises improve cardiovascular health, increase lung capacity, and help with weight loss.
""".strip()

Pattern Breakdown:

* ^: This matches the start of a string. It means the string must begin with what follows.
* Action:: This is a literal match. It means the string must have the text "Action:" at the beginning.
* (\w+):
    I The parentheses () define a capture group. This allows you to extract part of the string that matches this section.
    II \w+ matches one or more word characters (letters, digits, and underscores). This will capture a word that follows "Action:".
* :: This is a literal colon that separates the word matched by (\w+) from the rest of the string.
* (.*):
    I This is another capture group, where .* matches any character (.) zero or more times (*), which means it captures everything that comes after the second colon.

#### What does it do?
This regex is looking for a string that:

1. Starts with the text "Action:".
2. Has a word right after it, separated by a colon.
3. Then, after another colon, it captures everything that follow

In [17]:
import re
action_re = re.compile('^Action: (\w+): (.*)')

<>:2: SyntaxWarning: invalid escape sequence '\w'
<>:2: SyntaxWarning: invalid escape sequence '\w'
C:\Users\divak\AppData\Local\Programs\Python\Python311\ipykernel_16304\952804720.py:2: SyntaxWarning: invalid escape sequence '\w'
  action_re = re.compile('^Action: (\w+): (.*)')


In [18]:
# Sample strings
text1 = "Action: Move: North"
text2 = "Action: Jump: High"
text3 = "Error: Not an Action"

In [19]:
# Testing the pattern
match1 = action_re.match(text1)

In [20]:
match1

<re.Match object; span=(0, 19), match='Action: Move: North'>

In [21]:
match1.group(1)

'Move'

In [22]:
match1.group(2)

'North'

In [23]:
match3 = action_re.match(text3)

In [24]:
match3

In [25]:
match3.group(1)

AttributeError: 'NoneType' object has no attribute 'group'

In [26]:
import httpx
def wikipedia(query):
    response = httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": query,
        "format": "json"
    })
    return response.json()["query"]["search"][0]["snippet"]

In [27]:
import httpx
def simon_blog_search(query):
    response = httpx.get("https://datasette.simonwillison.net/simonwillisonblog.json", params={
        "sql": """
        select
          blog_entry.title || ': ' || substr(html_strip_tags(blog_entry.body), 0, 1000) as text,
          blog_entry.created
        from
          blog_entry join blog_entry_fts on blog_entry.rowid = blog_entry_fts.rowid
        where
          blog_entry_fts match escape_fts(:q)
        order by
          blog_entry_fts.rank
        limit
          1
        """.strip(),
        "_shape": "array",
        "q": query,
    })
    return response.json()[0]["text"]

In [28]:
def calculate(number):
    return eval(number)

In [29]:
calculate("2+2")

4

In [30]:
calculate("20+2*5")

30

In [31]:
known_actions = {
    "wikipedia": wikipedia,
    "calculate": calculate,
    "simon_blog_search": simon_blog_search,
    
}

In [32]:
def query(question,max_turns=5):
    i = 0
    bot = Chatbot(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        if actions:
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = f"Observation: {observation}"
        else:
            return result
    

In [34]:
prompt

'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop your output an Answer.\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\n\nYour available actions are:\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point\nsyntax if necessary\n\nwikipedia:\ne.g. wikipedia: Django\nReturns a summary from searching Wikipedia\n\nsimon_blog_search:\ne.g. simon_blog_search: Python\nSearch Simon\'s blog for that term\n\nExample session:\nQuestion: What is the capital of France?\nThought: I should look up France on Wikipedia\nAction: wikipedia: France\nPAUSE\n\nYou will be called again with this:\nObservation: France is a country. The capital is Paris.\n\nYou then output:\nAnswer: The capital of France is Paris\n\nPlease Note: if you get 

In [36]:
bot = Chatbot(prompt)

In [37]:
bot("hi how are you?")

'Thought: I was just waiting for a question, now I can respond with a greeting.\n\nAction: hello\nPAUSE\n\n(I will assume you wanted a friendly response.)'

In [38]:
bot("what is your job?")

"Thought: Hmm, I'm not sure if I should explain my programming role or just give a friendly response.\n\nAction: I am an assistant, here to help with your questions and tasks.\n\nPAUSE"

In [39]:
next_prompt="Tell me about quantum computing from Wikipedia."

In [40]:
next_prompt="Tell me about the current gdp of india from Wikipedia."

In [41]:
result=bot(next_prompt)

In [42]:
result

'Thought: I should find the current GDP of India on Wikipedia and return the information.\n\nAction: wikipedia: India GDP\nPAUSE'

In [43]:
actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]

In [44]:
actions

[<re.Match object; span=(0, 28), match='Action: wikipedia: India GDP'>]

In [45]:
action, action_input = actions[0].groups()

In [46]:
action

'wikipedia'

In [47]:
action_input

'India GDP'

In [48]:
query("hi")

Thought: The user said "hi", I should respond with a greeting.

Action: hello

PAUSE


'Thought: The user said "hi", I should respond with a greeting.\n\nAction: hello\n\nPAUSE'

In [49]:
query("how are you?")

Thought: The question is asking about my current state, a common greeting.

Action: i am good.

PAUSE

(Note: I don't have feelings or emotions like humans do, so I'm responding as is common in human conversation.)


"Thought: The question is asking about my current state, a common greeting.\n\nAction: i am good.\n\nPAUSE\n\n(Note: I don't have feelings or emotions like humans do, so I'm responding as is common in human conversation.)"

In [50]:
query("Fifteen * twenty five")

Thought: I need to calculate 15 times 25.

Action: calculate: 15 * 25
PAUSE
 -- running calculate 15 * 25
Observation: 375
Answer: 375


'Answer: 375'

In [51]:
query("Has Simon written about AI?")

Thought: I should search Simon's blog for mentions of AI.

Action: simon_blog_search: AI
PAUSE
 -- running simon_blog_search AI
Observation: It's OK to call it Artificial Intelligence: Update 9th January 2024: This post was clumsily written and failed to make the point I wanted it to make. I've published a follow-up, What I should have said about the term Artificial Intelligence which you should read instead.
My original post follows.


We need to be having high quality conversations about AI: what it can and can't do, its many risks and pitfalls and how to integrate it into society in the most beneficial ways possible.
Any time I write anything that mentions AI it's inevitable that someone will object to the very usage of the term.
Strawman: "Don't call it AI! It's not actually intelligent - it's just spicy autocomplete."
That strawman is right: it's not "intelligent" in the same way that humans are. And "spicy autocomplete" is actually a pretty good analogy for how a lot of these thi

'Thought: I should note that Simon has written about AI, and that there\'s a follow-up post about the term "Artificial Intelligence" that he recommends reading.\n\nAction: \nI\'ll take a note of the term "Artificial Intelligence" and also the recommended follow-up post. However, I will also note that the original post is available for reading if someone is interested.\n\nPAUSE'